Dana Rozenblum & Efrat Magidov
# Item-Item Top-N Recommendations

In this excercise we will implement a simple top-N recommender, evaluate the algorithms, and then call algorithms from the Surprise package. In top-N recommendations the algorithm is requested to produce a list of N items that the user will be interested in. 
In this particular execercise we will work with an escape room dataset.

First, let's load the dataset, which is already split by time into a training set and a test set:

In [2]:
import pandas as pd
train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]

We can take a look at the structure of the dataset:

In [3]:
train_set.head()
test_set.head()

,userID,itemID,rating,timestamp
index,,,,
69,431,366,9,14/09/2018
70,431,342,6,14/09/2018
71,431,362,7,14/09/2018
72,431,365,8,14/09/2018
73,433,201,10,14/09/2018


In [85]:
train_set.loc[train_set['userID']==431]

,userID,itemID,rating,timestamp
index,,,,
506,431,1,9,11/11/2015
507,431,2,7,11/11/2015
508,431,5,7,11/11/2015
509,431,8,10,11/11/2015
510,431,10,10,11/11/2015
...,...,...,...,...
39223,431,282,7,15/08/2018
39224,431,364,10,15/08/2018
39225,431,369,6,15/08/2018


## Part 1: Recommend Most Popular Items 

Now we can begin implementing the our first algorithm, that recommends to the user the list of most popular items. Although this is not a personalized approach, in many cases, this is not a bad idea - popular items are popular because everybody choose them, so there is a high likelihood that recommended popular items will be indeed chosen by the user.

In the code below, fill in the missing parts. The algorithm has a training method, where item popularity is computed, and a recommendation method, where the list of popular items.

In [169]:
import numpy as np

class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):
        self.train_set = train_set
        
        #1) Add code to set the item_ratings_sorted to the list of items in the training set, 
        #ordered by decreasing popularity (i.e., the number of users who have chosen an item)
        self.item_ratings_sorted = self.train_set.groupby(['itemID']).count().sort_values(by=['userID'],ascending=False)
        self.item_ratings_sorted['itemID'] = self.item_ratings_sorted.index
        self.item_ratings_sorted.drop(columns=['userID', 'rating','timestamp'],inplace=True)

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}
        already_ranked_items_by_user = self.train_set.groupby('userID')['itemID'].apply(list)
        
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            
            # Set the initial sorted list as the user result
            # Remove items that the user has already chosen in the training set (already_ranked_items_by_user)
            result[str(userID)] = \
            self.item_ratings_sorted.values[~np.in1d(self.item_ratings_sorted.values,already_ranked_items_by_user[userID])]
            
            # Take only top N items
            result[str(userID)] = result[str(userID)][0:top_n]
            result[str(userID)] = list(np.squeeze(result[str(userID)]))
        
        return result

    def clone(self):
        pass


Now we can call the most popular algorithm to deliver a list of reocmmendations. The code below prints the list of top 5 recommended items for user with ID 431.

In [170]:
popular = MostPopular()
popular.learn_model(train_set)
popular_recs = popular.get_top_n_recommendations(test_set,top_n=5)
print(popular_recs['431'])
assert popular_recs['431'] == [53, 26, 68, 85, 16], 'Wrong computation of popular items'

[53, 26, 68, 85, 16]


## Part 2 - Item-Item Recommendations

We now learn a slightly more sophisticated model, that uses item-item similarities. Given such a similarity score, we can recommend to a user items that are most similar to the items that the user has chosen in the past. One such useful similarity metric is the Jaccard coefficient. For two items i1 and i2, the Jaccard similarity is the number of users who have chosen both i1 and i2, divided by the number of users who have chosen either i1 or i2. That is, given the list of users who have chosen i1 and the list of users who have chosen i2, the Jaccard similarity is the intersection of the lists, divided by the union of the lists.

In practice, to expedite the recommendation process, and hence reduce online latency, we will compute the item-item co-occurence matrix in the model learning phase. Then, online, when recommendations are requested, we only need to compute for each item that the user has already chosen in the past, the Jaccard scores for the other items.

As the user has chosen several items in the past, we need to aggregate the Jaccard scores. That is, if the user has previously chosen i1 and i2, item i3 has two scores J(i1,i3) and J(i2,i3), and an aggregation of the scores is needed. There are two popular aggregation functions - sum and max. Empirically, max typically has better perfromance.

Fill in the missing parts in the code below.

In [201]:
from tqdm import tqdm
import numpy as np
import operator

class Jaccard:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None
        self.item_item_counts = dict()
        self.item_counts = None

    def learn_model(self, train_set):
        print('Started training')
        self.train_set = train_set
        self.item_counts =  self.train_set.groupby('itemID')['userID'].agg('count')
        
        maxpair = 0
        pbar = tqdm(total=len(train_set.userID.unique()))
        
        #Iterating over all item pairs is inefficient. It is better to iterate only over pairs of items that were chosen
        #together.
        #Instead, we will iterate over the users, and for each user, and each two items that the user has chosen, increase the
        #count
        for u in train_set.userID.unique():
            pbar.update(1)
            userData = self.train_set[self.train_set.userID == u]
            
            #4) For each pair of items in the user data - increase the counts in self.item_item_counts
            userItems = np.sort(userData.itemID.unique())
            nItems = userItems.size
            
            for first in range(nItems - 1):
                
                for second in range (first + 1, nItems, 1):
                    pair = (userItems[first], userItems[second])
                    
                    if pair in self.item_item_counts.keys():
                        self.item_item_counts[pair] += 1
                        
                    else:
                        self.item_item_counts[pair] = 1
        
        pbar.close()
        print('Done training')

    def get_top_n_recommendations(self, test_set, top_n):
        print('Started computing recommendations')
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)
        uniqueItems = self.train_set.itemID.unique()
        
        pbar = tqdm(total=len(test_set.userID.unique()))
        
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            pbar.update(1)
            result[str(userID)] = []
            #maxvalues will maintain for each potential item to recommend its highest Jaccard score.
            maxvalues = dict()
            
            #4) Iterate over the items that the user has already chosen.
            #For each such item compute its Jaccard correlation to other items based on the item_item_counts.
            #Aggregate into maxvalues using max.
            #Avoid recommending items that the user has already chosen in the training set.
            notChosen = uniqueItems[~np.in1d(uniqueItems,already_ranked_items_by_users[userID])]

            for nc in notChosen:
                scores = []
                
                for i in already_ranked_items_by_users[userID]:
                    first = min(nc, i)
                    second = max(nc, i)
                    
                    if (first,second) in self.item_item_counts.keys():
                        intersect = self.item_item_counts[(first,second)]
                    
                    else:
                        intersect = 0
                        
                    nc_count = len(self.train_set.loc[self.train_set['itemID']==nc])
                    i_count = len(self.train_set.loc[self.train_set['itemID']==i])
                    scores.append(intersect / (nc_count + i_count))
                
                maxvalues[nc] = np.max(scores)
                
            #Now we just take the top N items by decreasing Jaccard
            top_list = sorted(maxvalues.items(), key=lambda kv : -kv[1])
            i=0
            j=0
            while i < top_n and j < len(top_list):
                itemID = top_list[j][0]
                j = j + 1
                result[str(userID)].append(itemID)
                i = i + 1
            
        pbar.close()
        print('Done computing recommendations')
        return result

    
    def clone(self):
        pass


The code below trains a Jaccard model and generates recommendations. Training will take a while, as we need to iterate over all users, and for each user go over her items in quadratic time.

In [202]:
jaccard = Jaccard()
jaccard.learn_model(train_set)
jaccard_recs = jaccard.get_top_n_recommendations(test_set,top_n=5)










  0%|                                                                                        | 0/20197 [00:00<?, ?it/s]








  1%|▌                                                                           | 161/20197 [00:00<00:12, 1597.85it/s]










Started training


  2%|█▏                                                                          | 331/20197 [00:00<00:12, 1623.92it/s]








  2%|█▋                                                                          | 433/20197 [00:00<00:14, 1352.92it/s]








  3%|██▏                                                                         | 583/20197 [00:00<00:14, 1391.08it/s]








  3%|██▌                                                                         | 690/20197 [00:00<00:15, 1272.87it/s]








  4%|███                                                                         | 821/20197 [00:00<00:15, 1281.02it/s]








  5%|███▋                                                                        | 971/20197 [00:00<00:14, 1337.20it/s]








  5%|████                                                                       | 1108/20197 [00:00<00:14, 1343.97it/s]








  6%|████▌                                                                      | 1240/20197 [00:00<00:1

 84%|██████████████████████████████████████████████████████████████            | 16945/20197 [00:10<00:01, 1631.75it/s]








 85%|██████████████████████████████████████████████████████████████▋           | 17109/20197 [00:11<00:01, 1608.90it/s]








 86%|███████████████████████████████████████████████████████████████▎          | 17271/20197 [00:11<00:01, 1608.73it/s]








 86%|███████████████████████████████████████████████████████████████▉          | 17446/20197 [00:11<00:01, 1645.39it/s]








 87%|████████████████████████████████████████████████████████████████▌         | 17619/20197 [00:11<00:01, 1669.76it/s]








 88%|█████████████████████████████████████████████████████████████████▏        | 17787/20197 [00:11<00:01, 1664.23it/s]








 89%|█████████████████████████████████████████████████████████████████▊        | 17954/20197 [00:11<00:01, 1608.23it/s]








 90%|██████████████████████████████████████████████████████████████████▍       | 18117/20197 [00:11<00:0

Done training
Started computing recommendations











  0%|                                                                                          | 0/227 [00:00<?, ?it/s]








  1%|▋                                                                                 | 2/227 [00:18<34:51,  9.30s/it]








  1%|█                                                                                 | 3/227 [00:37<45:23, 12.16s/it]








  2%|█▍                                                                                | 4/227 [00:40<35:02,  9.43s/it]








  2%|█▊                                                                                | 5/227 [00:40<24:49,  6.71s/it]








  3%|██▏                                                                               | 6/227 [00:43<20:36,  5.60s/it]








  3%|██▌                                                                               | 7/227 [01:03<35:45,  9.75s/it]








  4%|██▉                                                                               | 8/227 

 46%|█████████████████████████████████████                                           | 105/227 [12:32<08:12,  4.04s/it]








 47%|█████████████████████████████████████▎                                          | 106/227 [12:33<05:55,  2.94s/it]








 47%|█████████████████████████████████████▋                                          | 107/227 [12:57<18:49,  9.41s/it]








 48%|██████████████████████████████████████                                          | 108/227 [13:07<18:59,  9.58s/it]








 48%|██████████████████████████████████████▍                                         | 109/227 [13:08<13:37,  6.93s/it]








 48%|██████████████████████████████████████▊                                         | 110/227 [13:08<09:41,  4.97s/it]








 49%|███████████████████████████████████████                                         | 111/227 [13:11<08:25,  4.36s/it]








 49%|███████████████████████████████████████▍                                        | 112/227 [13:12<06

 92%|█████████████████████████████████████████████████████████████████████████▋      | 209/227 [20:59<01:26,  4.79s/it]








 93%|██████████████████████████████████████████████████████████████████████████      | 210/227 [21:00<00:59,  3.47s/it]








 93%|██████████████████████████████████████████████████████████████████████████▎     | 211/227 [21:00<00:40,  2.55s/it]








 93%|██████████████████████████████████████████████████████████████████████████▋     | 212/227 [21:01<00:28,  1.90s/it]








 94%|███████████████████████████████████████████████████████████████████████████     | 213/227 [21:12<01:08,  4.87s/it]








 94%|███████████████████████████████████████████████████████████████████████████▍    | 214/227 [21:15<00:55,  4.25s/it]








 95%|███████████████████████████████████████████████████████████████████████████▊    | 215/227 [21:16<00:39,  3.31s/it]








 95%|████████████████████████████████████████████████████████████████████████████    | 216/227 [21:18<00

Done computing recommendations


In [205]:
jaccard_recs

{'431': [296, 297, 298, 299, 300],
 '433': [152, 43, 40, 44, 14],
 '4463': [254, 68, 229, 336, 195],
 '12663': [74, 204, 195, 75, 89],
 '4501': [75, 54, 203, 8, 4],
 '4502': [269, 266, 346, 287, 317],
 '582': [40, 331, 38, 17, 39],
 '4516': [331, 68, 229, 275, 38],
 '660': [74, 93, 287, 348, 267],
 '671': [182, 10, 4, 277, 370],
 '681': [10, 3, 8, 66, 4],
 '697': [8, 126, 10, 21, 18],
 '765': [204, 10, 8, 66, 4],
 '4809': [242, 2, 126, 4, 120],
 '12684': [331, 89, 204, 274, 84],
 '4957': [269, 346, 266, 285, 287],
 '4993': [68, 204, 149, 8, 4],
 '5397': [2, 126, 4, 120, 204],
 '5517': [331, 2, 336, 195, 89],
 '5518': [269, 270, 40, 277, 346],
 '5534': [283, 336, 269, 13, 160],
 '5557': [358, 293, 193, 279, 296],
 '5559': [175, 2, 86, 23, 109],
 '5640': [371, 293, 367, 331, 268],
 '5713': [54, 10, 4, 1, 8],
 '5731': [119, 95, 60, 67, 145],
 '12839': [283, 68, 229, 317, 145],
 '12940': [274, 281, 195, 336, 331],
 '12979': [274, 281, 195, 336, 331],
 '13013': [254, 319, 331, 72, 39],
 '19

As a side note - as computing the ite-item counts takes a while (especially with Python), we are using here the progress bar from the tqdm package (https://pypi.org/project/tqdm/). You need to install tqdm, or remove the progress bar, which would of course is not needed for the algorithm to run.

## Part 3 - Comparing the Algorithms 

We now want to compare the recommendation lists to see which one is better. In top-N recommendations it is popular to computer the Precision@N metric - the portion of recommended items that were chosen by users in the test set. This is typically a reasonable metric for real systems, where one wants to optimize the number of recommended items that are chosen.

We compute Precision@N by comparing the number of recommendations chosen by the users, divided by the number of overall recommendations.

Fill in the missing parts in the code below:

In [209]:
def compute_precision(test_set, recommendations):
    #hits is the number of items that were recommended and chosen
    hits = 0
    #recs is the total number of recommended items
    recs = 0
    
    for u in test_set.userID.unique():
        userData = test_set[test_set.userID == u]
        userRecs = recommendations.get(str(u))
        
        #5) Compute here the number of hits. Update hits and recs accordingly.
        hits += len(set(userData["itemID"]) & set(userRecs))
        recs += len(userRecs)
        
    return hits / recs
        

In [210]:
p1 = compute_precision(test_set, jaccard_recs)
p2 = compute_precision(test_set, popular_recs)
print("Jaccard=", p1, "  Popularity=", p2)

Jaccard= 0.03700440528634361   Popularity= 0.027312775330396475


The precision values for this dataset may seem pretty low, but this is typical for many top-N problems. It is important not to compute metrics that hide the true values, such as AUC, but to acknowledge the perfromance of the system in the application.

## Part 4 - Calling Algorithms from the Surprise Package

There are many existing recommendation algorithms available. We will now see how we can call algorithms from the Surprise package. 

The code below adds a wrapper around the algorithm to transform the resulting recommendations into our desired format.

#### NOTE: 
To run the code below you first have to install _surprise_ (http://surpriselib.com/). surprise requires scipy >=1.0, so update if needed.

To install: pip install scikit-surprise or, if you're using anaconda:  conda install -c conda-forge scikit-surprise).  

In [212]:
import sys, string, os
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import operator
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold
from surprise.prediction_algorithms import *



class SurpriseRecMethod():

    #method will be the specific Surprise algorithm that we will call
    def __init__(self, method):
        self.method = method

    def fit(self, train_set):
        self.train_set = train_set


    def get_top_n_recommendations(self, test_set, top_n):
        self.test_set = test_set

        #Surprise requires a slightly different input data format, so we use two different CSVs
        test_path_tmp = "resources//test_file.csv"
        train_path_tmp = "resources//train_file.csv"

        self.train_set.to_csv(train_path_tmp, index=False, header=False)
        self.test_set.to_csv(test_path_tmp, index=False, header=False)

        fold_files = [(train_path_tmp, test_path_tmp)]
        reader = Reader(rating_scale=(1, 10), line_format='user item rating', sep=',')
        data = Dataset.load_from_folds(fold_files, reader=reader)

        for trainset, testset in PredefinedKFold().split(data):
            self.method.fit(trainset)

        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        recommendations = {}
        pbar = tqdm(total=len(self.test_set.userID.unique()))
        for userID in self.test_set.userID.unique():
            pbar.update(1)

            if userID not in self.train_set.userID.unique():
                recommendations[str(userID)] = []
                continue

            items_expected_ranking = {}
            for itemID in self.train_set.itemID.unique():
                if itemID in already_ranked_items_by_users[userID]:
                    continue
                #We call here the specific Surprise method that we use for this model
                #The method predicts a score for a given item
                predicted = self.method.predict(str(userID), str(itemID), clip=False)
                items_expected_ranking[itemID] = predicted[3]
                
            #Now we just sort by decreasing scores and take the top N
            sorted_predictions = sorted(items_expected_ranking.items(), key=operator.itemgetter(1))
            sorted_predictions.reverse()
            sorted_predictions = [x[0] for x in sorted_predictions]
            user_recommendations = sorted_predictions[:top_n]
            recommendations[str(userID)] = user_recommendations
        pbar.close()
        return recommendations


The code below calls the package with the SlopeOne algorithm.

In [213]:
modelSlopeOne = SurpriseRecMethod(SlopeOne())
modelSlopeOne.fit(train_set)
recSlopeOne = modelSlopeOne.get_top_n_recommendations(test_set, 5)
p3 = compute_precision(test_set,recSlopeOne)










  0%|                                                                                          | 0/227 [00:00<?, ?it/s]








  1%|█                                                                                 | 3/227 [00:00<00:07, 28.65it/s]








  4%|██▉                                                                               | 8/227 [00:00<00:06, 32.75it/s]








  6%|████▉                                                                            | 14/227 [00:00<00:05, 36.88it/s]








  9%|███████▏                                                                         | 20/227 [00:00<00:05, 40.63it/s]








 11%|████████▌                                                                        | 24/227 [00:00<00:05, 34.69it/s]








 12%|█████████▉                                                                       | 28/227 [00:00<00:05, 33.36it/s]








 15%|████████████▏                                                                    | 34/227 

The code below calls the package with a nearest neighbor user-item recommendation method.

In [214]:
modelKNNUser = SurpriseRecMethod(KNNBasic(sim_options={'name': 'cosine', 'user_based': True}))
modelKNNUser.fit(train_set)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_set, 5)
p4 = compute_precision(test_set,recKNNUser)

Computing the cosine similarity matrix...
Done computing similarity matrix.











  0%|                                                                                          | 0/227 [00:00<?, ?it/s]








  1%|█                                                                                 | 3/227 [00:00<00:10, 20.46it/s]








  2%|█▊                                                                                | 5/227 [00:00<00:13, 17.00it/s]








  3%|██▏                                                                               | 6/227 [00:00<00:16, 13.66it/s]








  4%|██▉                                                                               | 8/227 [00:00<00:17, 12.75it/s]








  4%|███▎                                                                              | 9/227 [00:00<00:19, 11.42it/s]








  5%|███▉                                                                             | 11/227 [00:00<00:18, 11.84it/s]








  6%|████▋                                                                            | 13/227 

 93%|██████████████████████████████████████████████████████████████████████████▎     | 211/227 [00:12<00:00, 17.59it/s]








 94%|███████████████████████████████████████████████████████████████████████████     | 213/227 [00:12<00:00, 17.33it/s]








 95%|███████████████████████████████████████████████████████████████████████████▊    | 215/227 [00:12<00:00, 16.92it/s]








 96%|████████████████████████████████████████████████████████████████████████████▍   | 217/227 [00:12<00:00, 16.61it/s]








 96%|█████████████████████████████████████████████████████████████████████████████▏  | 219/227 [00:12<00:00, 16.95it/s]








 97%|█████████████████████████████████████████████████████████████████████████████▉  | 221/227 [00:12<00:00, 16.84it/s]








 98%|██████████████████████████████████████████████████████████████████████████████▌ | 223/227 [00:13<00:00, 16.97it/s]








 99%|███████████████████████████████████████████████████████████████████████████████▎| 225/227 [00:13<00

Let us look at the results of all algorithms together:

In [215]:
pd.DataFrame.from_dict({'Jaccard':p1,'Popularity':p2,'SlopeOne':p3,'User KNN':p4}, orient='index',columns=['Precision'])

,Precision
Jaccard,0.037004
Popularity,0.027313
SlopeOne,0.032599
User KNN,0.075771


Try the NMF (non-negative matrix factorization) algorithms from the package - https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF.


In [219]:
#Your code here
modelNMF = SurpriseRecMethod(NMF())
modelNMF.fit(train_set)
recNMF = modelNMF.get_top_n_recommendations(test_set, 5)
p5 = compute_precision(test_set,recNMF)










  0%|                                                                                          | 0/227 [00:00<?, ?it/s]








  3%|██▏                                                                               | 6/227 [00:00<00:03, 57.19it/s]








  6%|████▉                                                                            | 14/227 [00:00<00:03, 61.71it/s]








 10%|███████▊                                                                         | 22/227 [00:00<00:03, 64.13it/s]








 13%|██████████▎                                                                      | 29/227 [00:00<00:03, 64.26it/s]








 16%|█████████████▏                                                                   | 37/227 [00:00<00:02, 66.14it/s]








 19%|███████████████▎                                                                 | 43/227 [00:00<00:03, 60.00it/s]








 22%|█████████████████▍                                                               | 49/227 

In [222]:
pd.DataFrame.from_dict({'Jaccard':p1,'Popularity':p2,'SlopeOne':p3,'User KNN':p4,'NMF':p5}, orient='index',columns=['Precision'])

,Precision
Jaccard,0.037004
Popularity,0.027313
SlopeOne,0.032599
User KNN,0.075771
NMF,0.039648


For this particular dataset, the user nearest neighbor approach worked best. Hence, should we need to choose a method to put online, we should go with this method.